In [2]:
import pandas as pd
import os
import sys
import csv
import numpy as np
import ast
from spacy.matcher import PhraseMatcher
from spacy import displacy

# Get the parent directory of the current directory
parent_dir = os.path.abspath(os.path.join(os.getcwd(), '..'))

# Add the parent directory to the Python path
sys.path.append(parent_dir)
import myutils
from config import Config

/Users/felix/opt/anaconda3/envs/thesis3.9/lib/python3.9/site-packages/requests/__init__.py:109: RequestsDependencyWarning: urllib3 (1.26.14) or chardet (None)/charset_normalizer (3.0.1) doesn't match a supported version!
  warnings.warn(


In [3]:
config = Config()
rubrics = myutils.load_rubrics('../' + config.PATH_RUBRIC)

# are long rubrics an issue?
lens = {}
for k,v in rubrics.items():
    lens[k] = len(v['key_element'].tolist())

sorted_lens = sorted(lens.items(), key=lambda x: x[1], reverse=True)
print(sorted_lens)

[('8.1_MM', 18), ('5.12', 16), ('2.4', 16), ('10.1_TC', 11), ('6.1_IPP', 10), ('10.2_TC', 7), ('10.3_TC', 7), ('5.7', 7), ('2.1_DLL_v1.1', 6), ('12.1_PE', 6), ('6.2_IPP', 6), ('6.3_IPP', 6), ('12.2_PE', 5), ('6.3', 5), ('1.6', 5), ('7', 4), ('1', 4), ('3', 4), ('4.1_LM_v1.0', 3), ('8.2_MM', 3), ('4.2_LM_v1.0', 3), ('6', 2), ('4', 2), ('4.3_LM', 2), ('8.3_MM', 2), ('4.13', 2), ('4.3', 2), ('2.3_DLL_v1.1', 2), ('2.2_DLL', 2), ('12.3_PE', 2), ('8', 2), ('5.11', 1)]


In [4]:
# how much similar spans do we predict?

duplicates = []
differences = []
lengths = []
# Open the CSV file and read its contents
with open('../results/grading_final_decision_tree_span_prediction_regression/final_prediction.csv', 'r') as csv_file:
    reader = csv.DictReader(csv_file)

    # Iterate over each row in the CSV file
    for row in reader:
        # Extract the justification_cues column as a string
        justification_cues_str = row['justification_cues']
        
        # Convert the string representation of the list of tuples to an actual list
        justification_cues = eval(justification_cues_str)
        
        duplicates.append(len(justification_cues) != len(set(justification_cues)))
        differences.append(len(justification_cues) - len(set(justification_cues)))
        lengths.append(len(justification_cues))
print(len([d for d in duplicates if d==True]))
print(np.average(differences))
        

411
1.784829721362229


In [5]:
converters = {
    'justification_cues': ast.literal_eval,
    'scoring_vectors': ast.literal_eval,
    'justification_cue_spans': ast.literal_eval,
}

In [6]:
# calculate metrics without unseen questions
from sklearn.metrics import mean_squared_error
from sklearn.metrics import classification_report
from sklearn.metrics import cohen_kappa_score

german_ids = [str(i) for i in range(0,9)]

unseen_question_ids = ['2', '5', '1.2', '11.1_SDN', '11.2_SDN', '11.3_SDN', '11.4_SDN']

In [7]:
file_paths = ['../results/grading_final_decision_tree_span_prediction_regression/final_prediction.csv',
              '../results/grading_final_decision_tree_token_classification_regression/final_prediction.csv',
              '../results/grading_final_summation_span_prediction_regression/final_prediction.csv',
              '../results/grading_final_summation_token_classification_regression/final_prediction.csv',
             ] 

In [8]:
for file_path in file_paths:
    y_true_de, y_pred_de = [], []
    y_true_en, y_pred_en = [], []
    predictions = pd.read_csv(file_path, converters=converters)
    for i, p in predictions.iterrows():
        if p['question_id'] not in unseen_question_ids:
            if p['question_id'] in german_ids:
                y_true_de.append(float(p['score']))
                y_pred_de.append(float(p['y_pred']))
            else:
                y_true_en.append(float(p['score']))
                y_pred_en.append(float(p['y_pred']))
    print(file_path.split('/')[2])
    print('DE', mean_squared_error(y_true_de, y_pred_de, squared=True))
    print('EN', mean_squared_error(y_true_en, y_pred_en, squared=True))
    y_true_de = [str(i) for i in y_true_de]
    y_pred_de = [str(i) for i in y_pred_de]
    y_true_en = [str(i) for i in y_true_en]
    y_pred_en = [str(i) for i in y_pred_en]
    print('DE QWK', cohen_kappa_score(y_true_de, y_pred_de, weights='quadratic'))
    print('EN QWK', cohen_kappa_score(y_true_en, y_pred_en, weights='quadratic'))

grading_final_decision_tree_span_prediction_regression
DE 0.08902214022140222
EN 0.07320833333333333
DE QWK 0.5955458464631624
EN QWK 0.4212823236835116
grading_final_decision_tree_token_classification_regression
DE 0.16576337638376384
EN 0.093625
DE QWK 0.13877081542845704
EN QWK 0.2800901184919762
grading_final_summation_span_prediction_regression
DE 0.43219557195571956
EN 0.5784166666666667
DE QWK 0.07812420583344826
EN QWK 0.035800488911631234
grading_final_summation_token_classification_regression
DE 0.4481088560885609
EN 0.5874583333333333
DE QWK 0.05816434764099743
EN QWK 0.05894317036811114


In [9]:
file_paths = ['../results/grading_final_decision_tree_span_prediction_classification/final_prediction.csv',
              '../results/grading_final_decision_tree_token_classification_classification/final_prediction.csv',
              '../results/grading_final_summation_span_prediction_classification/final_prediction.csv',
              '../results/grading_final_summation_token_classification_classification/final_prediction.csv',
             ] 

In [10]:
for file_path in file_paths:
    y_true_de, y_pred_de = [], []
    y_true_en, y_pred_en = [], []
    predictions = pd.read_csv(file_path, converters=converters)
    for i, p in predictions.iterrows():
        if p['question_id'] not in unseen_question_ids:
            if p['question_id'] in german_ids:
                y_true_de.append(p['class'])
                y_pred_de.append(p['y_pred'])
            else:
                y_true_en.append(p['class'])
                y_pred_en.append(p['y_pred'])
    print(file_path.split('/')[2])
    print('DE', classification_report(y_true_de, y_pred_de, digits=3))
    print('EN', classification_report(y_true_en, y_pred_en, digits=3))
    print('DE QWK', cohen_kappa_score(y_true_de, y_pred_de, weights='quadratic'))
    print('EN QWK', cohen_kappa_score(y_true_en, y_pred_en, weights='quadratic'))
    print(20*'-')

grading_final_decision_tree_span_prediction_classification
DE                  precision    recall  f1-score   support

        CORRECT      0.651     0.689     0.670       103
      INCORRECT      0.714     0.213     0.328        47
PARTIAL_CORRECT      0.649     0.793     0.714       121

       accuracy                          0.653       271
      macro avg      0.671     0.565     0.570       271
   weighted avg      0.661     0.653     0.630       271

EN                  precision    recall  f1-score   support

        CORRECT      0.827     0.758     0.791       240
      INCORRECT      0.294     0.238     0.263        21
PARTIAL_CORRECT      0.580     0.702     0.635       114

       accuracy                          0.712       375
      macro avg      0.567     0.566     0.563       375
   weighted avg      0.722     0.712     0.714       375

DE QWK 0.4682854195909948
EN QWK 0.4702357755261106
--------------------
grading_final_decision_tree_token_classification_classific

/Users/felix/opt/anaconda3/envs/thesis3.9/lib/python3.9/site-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/Users/felix/opt/anaconda3/envs/thesis3.9/lib/python3.9/site-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/Users/felix/opt/anaconda3/envs/thesis3.9/lib/python3.9/site-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier

grading_final_summation_span_prediction_classification
DE                  precision    recall  f1-score   support

        CORRECT      1.000     0.029     0.057       103
      INCORRECT      0.199     1.000     0.332        47
PARTIAL_CORRECT      0.312     0.083     0.131       121

       accuracy                          0.221       271
      macro avg      0.504     0.371     0.173       271
   weighted avg      0.554     0.221     0.137       271

EN                  precision    recall  f1-score   support

        CORRECT      0.744     0.121     0.208       240
      INCORRECT      0.075     0.762     0.137        21
PARTIAL_CORRECT      0.407     0.439     0.422       114

       accuracy                          0.253       375
      macro avg      0.408     0.440     0.256       375
   weighted avg      0.604     0.253     0.269       375

DE QWK -0.08564610600911138
EN QWK 0.10210185029906649
--------------------
grading_final_summation_token_classification_classification

In [16]:
file_path = '../results/grading_final_decision_tree_span_prediction_regression/final_prediction.csv'
predictions_span = pd.read_csv(file_path, converters=converters)
file_path = '../results/grading_final_decision_tree_token_classification_regression/final_prediction.csv'
predictions_token = pd.read_csv(file_path, converters=converters)

In [45]:
def display_ents(predictions):
    def on_match(matcher, doc, id, matches):
        ents = []
        for m in matches:
            start = m[0]
            end = m[1]
            ents.append({
                'label': 'CUE',
                'span_start': start,
                'span_end': end
            })
        return matches

    def merge_spans(spans):
        sorted_spans = sorted(spans, key=lambda x: x[1])
        merged_spans = []
        for span in sorted_spans:
            if not merged_spans:
                merged_spans.append(span)
            else:
                last_span = merged_spans[-1]
                if last_span[2] >= span[1]:
                    merged_span = (last_span[0], last_span[1], max(last_span[2], span[2]))
                    merged_spans[-1] = merged_span
                else:
                    merged_spans.append(span)
        return merged_spans

    for i, d in predictions.iterrows():
        text = config.nlp(d['text'])
        matcher = PhraseMatcher(config.nlp.vocab)
        spans = []
        for jus_cue in d['justification_cue_spans']:
            print(jus_cue)
            for j in jus_cue:
                if len(j):
                    span = config.nlp(j)
                    spans.append(span)
        matcher.add('CUE',spans, on_match=on_match)
        matches = matcher(text)
        print(matches)
        matches = merge_spans(matches)
        text.ents = matches
        print(d['question_id'])
        displacy.render(text, style="ent", options={'colors': {'CUE': '#F5F503'}})
        print('True score:', d['score'], 'Predicted score:', d['y_pred'])

In [46]:
#display_ents(predictions_token)

In [47]:
display_ents(predictions_span)

['Das alles gut zu', 'Das alles gut zu']
[(13999982436800569226, 0, 4)]
8


True score: 0.0 Predicted score: 0.125
['(z.B.Sauberkeit', 'Waren bei deinem Stationsbesuch Handschuhe oder Papierhandtücher (an der Zapfsäule) oder Handdesinfektionsmittel (am Shopeingang) verfügbar, sodass du deine Hände sauber halten konntest? Bist du mit der Warenpräsentation des Shops innen und außen zufrieden']
[(13999982436800569226, 47, 86), (13999982436800569226, 87, 90)]
8


True score: 1.0 Predicted score: 0.875
['Arbeitskleidung zulässig? Wurdest du von allen Mitarbeitern, die dich im Shop bedienten, begrüßt und verabschiedet? Es zählen auch non-verbale Gesten, wie z.B. winken oder nicken. Haben sich die Mitarbeiter im Shop, die dich bedienten, um eine persönlichere Begrüßung oder Verabschiedung bemüht (z.B. Blickkontakt, Lächeln)? Wie zufrieden warst du mit der Freundlichkeit und Gastfreundschaft der Mitarbeiter des Shops, die dich bedienten? (5 = sehr zufrieden bis 1 = sehr unzufrieden) Wie zufrieden warst du mit der Bedienung durch die Mitarbeiter? (5 = “Sehr zufrieden” und 1 = “Sehr unzufrieden”)', 'Arbeitskleidung zulässig? Wurdest du von allen Mitarbeitern, die dich im Shop bedienten, begrüßt und verabschiedet? Es zählen auch non-verbale Gesten, wie z.B. winken oder nicken. Haben sich die Mitarbeiter im Shop, die dich bedienten, um eine persönlichere Begrüßung oder Verabschiedung bemüht (z.B. Blickkontakt']
[(13999982436800569226, 0, 58), (13999982

True score: 0.0 Predicted score: 0.625
['Dann ob der Shop sauber ist und die Gänge nicht zugestellt sind', 'Ob die Regale gut gefüllt sind und das warenangebot gut sortiert ist']
[(13999982436800569226, 0, 12), (13999982436800569226, 13, 25)]
8


True score: 1.0 Predicted score: 0.875
['Gepflegter Außenbereich mit den dortigen Einrichtungen, Vorhandensein von Handschuhen, Papierhandtüchern oder Desinfektionsmittel sowie Warenpräsentation', 'Gepflegter Außenbereich mit den dortigen Einrichtungen, Vorhandensein von Handschuhen, Papierhandtüchern oder Desinfektionsmittel sowie Warenpräsentation']
[(13999982436800569226, 0, 16)]
8


...

True score: 1.0 Predicted score: 1.0
['Since the system reached an equilibrium the probabilities do not change anymore ( dp_n(t)/dt = 0 ). So it can be assumed that the queue is emptied by one package each second on average (10 served - 9 arrived). Now assuming that the queue is full at obvservation start', 'Since the system reached an equilibrium the probabilities do not change anymore ( dp_n(t)/dt = 0 ).', 'Since the system reached an equilibrium the probabilities do not change anymore ( dp_n(t)/dt = 0 ).', 'Since the system reached an equilibrium the probabilities do not change anymore ( dp_n(t)/dt = 0 ). So it can be assumed that the queue is emptied by one package each second on average (10 served - 9 arrived). Now assuming that the queue is full at obvservation start, it will be empty after 10 seconds', 'it will be empty after 10 seconds']
[(13999982436800569226, 0, 18), (13999982436800569226, 55, 62)]
12.2_PE


True score: 0.0 Predicted score: 0.0
['we calculated the probability for the system to be in state 10, i.e. the probability that the system is full', 'we calculated the probability for the system to be in state 10, ', 'p_10=((1-0.9)*0.9^10)/(1-0.9^11) = 0.051 Next, we calculated the counter probability of p_10 as we actually want to know with which probability the system is not in state 10. 1-p_10 = 1-0.051 = 0.949 To get the number of seconds the system is not in state 10, we calculated 60s * 0.949 = 56.94s as we measure the system for 60 seconds', '56.94s as we measure the system for 60 seconds', '1-p_10 = 1-0.051 = 0.949 To get the number of seconds the system is not in state 10, we calculated 60s * 0.949 = 56.94s as we measure the system for 60 seconds']
[(13999982436800569226, 45, 58), (13999982436800569226, 45, 66), (13999982436800569226, 67, 143), (13999982436800569226, 105, 143), (13999982436800569226, 134, 143)]
12.2_PE


True score: 1.0 Predicted score: 1.0
['The probability is about 5%. So it’s likely that the system will most likely have the full minute less than 10 packets', 'The probability is about 5%. So it’s likely that the system will most likely have the full minute less than 10 packets', 'We can calculate probability for that using the above-mentioned utilization', 'We can calculate probability for that using the above-mentioned utilization', 'In this scenario the utilization is quite high with 90%. However we want to know the time that the buffer has less than 10 entries. This means that not the case if the buffer is full. We can calculate probability for that using the above-mentioned utilization']
[(13999982436800569226, 8, 60), (13999982436800569226, 48, 60), (13999982436800569226, 61, 86)]
12.2_PE


True score: 0.25 Predicted score: 1.0
['p = 9/10 = 0.9 The expected Number of Customers in the System are p/1-p = 0.9/0.1 = 9.', 'p = 9/10 = 0.9 The expected Number of Customers in the System are p/1-p = 0.9/0.1 = 9. I expect that the queue always has less than 10 packets waiting in it', 'p = 9/10 = 0.9 The expected Number of Customers in the System are p/1-p = 0.9/0.1 = 9.', 'p = 9/10 = 0.9 The expected Number of Customers in the System are p/1-p = 0.9/0.1 = 9.', 'p = 9/10 = 0.9 The expected Number of Customers in the System are p/1-p = 0.9/0.1 = 9.']
[(13999982436800569226, 0, 22), (13999982436800569226, 0, 36)]
12.2_PE


True score: 0.0 Predicted score: 1.0


In [ ]:
training_data = myutils.load_json('../data/training_dataset.json')

In [ ]:
training_data

In [ ]:
from collections import Counter
import matplotlib.pyplot as plt

# Count the occurrences of each question_id
question_counts = Counter(q['question_id'] for q in training_data)

# Plot the histogram
plt.bar(question_counts.keys(), question_counts.values())
plt.xlabel('Question ID')
plt.xticks(rotation=90)
plt.ylabel('Count')
plt.show()

In [ ]:
from collections import Counter
import matplotlib.pyplot as plt

# Count the occurrences of each question_id
for l, c in zip(['CORRECT', 'PARTIAL_CORRECT', 'INCORRECT'],  ['green', 'orange', 'red']):
    question_counts = Counter(q['question_id'] for q in training_data if q['label']==l)
    question_counts = sorted(question_counts.items(), key=lambda x: x[0])
    question_counts = {k: v for k, v in question_counts}
    # Plot the histogram
    
    plt.bar(question_counts.keys(), question_counts.values(), color=c)
    plt.title('Training Data Distribution for class: ' + str(l) )
    plt.xlabel('Question ID')
    plt.xticks(rotation=90)
    plt.ylim((0,200))
    plt.ylabel('Count')
    plt.show()

In [ ]:
import numpy as np
from scipy.stats import pearsonr

rubric_length = np.array([4, 3, 2, 2, 4, 2, 16, 1, 2, 5, 7, 6, 2, 18, 3, 5, 16, 2, 11, 10, 6, 2, 7, 2, 2, 2, 6, 4, 3, 6, 7, 2])
macro_f1 = np.array([0.224, 0.133, 0.167, 0.017, 0.208, 0.037, 1.000, 0.489, 0.275, 0.242, 0.052, 0.483, 0.000, 0.098, 0.484, 0.154, 0.240, 0.180, 0.156, 1.000, 0.570, 0.047, 0.400, 0.222, 0.478, 0.050, 0.393, 0.181, 0.026, 0.190, 0.228, 0.229])
weighted_f1 = np.array([0.29, 0.28, 0.63, 0.04, 0.44, 0.18, 1.00, 0.94, 0.74, 0.49, 0.17, 0.97, 0.00, 0.40, 0.97, 0.36, 0.51, 0.32, 0.58, 1.000, 0.83, 0.05, 0.50, 0.40, 0.88, 0.09, 0.44, 0.79, 0.01, 0.45, 0.38, 0.85])
acc = np.array([0.25, 0.241, 0.61, 0.019, 0.415, 0.103, 1, 0.957, 0.7, 0.5, 0.182, 0.933, 0, 0.313, 0.938, 0.375, 0.529, 0.385, 0.583, 1, 0.857, 0.1, 0.5, 0.444, 0.917, 0.071, 0.474, 0.792, 0.077, 0.412, 0.357, 0.786])


weighted_corr_coef, weighted_p_value = pearsonr(weighted_f1, rubric_length)
print("Weighted Pearson correlation coefficient:", weighted_corr_coef)
print("Weighted p-value:", weighted_p_value)

macro_corr_coef, macro_p_value = pearsonr(macro_f1, rubric_length)
print("Macro-F1 Pearson correlation coefficient:", macro_corr_coef)
print("Macro-F1 p-value:", macro_p_value)

acc_corr_coef, acc_p_value = pearsonr(acc, rubric_length)
print("Macro-F1 Pearson correlation coefficient:", acc_corr_coef)
print("Macro-F1 p-value:", acc_p_value)

